## **Reptile Following MouseCallback Animation (Dynamically)**

In [ ]:
import cv2
import numpy as np
import math

# --- Parameters ---
num_segments = 20      # Number of body parts
segment_length = 20    # Distance between joints
speed = 0.2            # Smoothness factor

# Create a blank window
width, height = 800, 600
window_name = "Reptile Following Mouse"
cv2.namedWindow(window_name)

# Initialize segment positions (starting from center)
segments = np.array([[width//2, height//2] for _ in range(num_segments)], dtype=np.float32)

# Track mouse position
mouse_pos = [width//2, height//2]

def mouse_event(event, x, y, flags, param):
    global mouse_pos
    if event == cv2.EVENT_MOUSEMOVE:
        mouse_pos = [x, y]

cv2.setMouseCallback(window_name, mouse_event)

while True:
    frame = np.zeros((height, width, 3), dtype=np.uint8)

    # Head follows the mouse
    dx = mouse_pos[0] - segments[0][0]
    dy = mouse_pos[1] - segments[0][1]
    angle = math.atan2(dy, dx)
    segments[0][0] += math.cos(angle) * speed * segment_length
    segments[0][1] += math.sin(angle) * speed * segment_length

    # Each segment follows the previous one
    for i in range(1, num_segments):
        dx = segments[i-1][0] - segments[i][0]
        dy = segments[i-1][1] - segments[i][1]
        angle = math.atan2(dy, dx)
        segments[i][0] = segments[i-1][0] - math.cos(angle) * segment_length
        segments[i][1] = segments[i-1][1] - math.sin(angle) * segment_length

    # Draw the creature
    for i in range(num_segments - 1):
        color = (255 - i*10, 100 + i*5, 255)  # Gradient color
        cv2.line(frame,
                 (int(segments[i][0]), int(segments[i][1])),
                 (int(segments[i+1][0]), int(segments[i+1][1])),
                 color, 2)
        cv2.circle(frame, (int(segments[i][0]), int(segments[i][1])), 3, (200, 200, 255), -1)

    cv2.putText(frame, "Move your mouse...", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow(window_name, frame)

    if cv2.waitKey(10) & 0xFF == 27:  # ESC to exit
        break

cv2.destroyAllWindows()